In [1]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import time

# List of URLs to start scraping from
start_urls = [
    'https://baylegal.org/',
    'https://www.lawhelpca.org/',
    'https://www.cuav.org/',
    'https://www.americanbar.org/groups/domestic_violence/',
    'https://www.cpedv.org/',
    'https://www.cpedv.org/bay-area-region',
    'http://www.feminist.org/911/crisis.html',
    'https://www.columbusfamilylaw.org/news/understanding-the-signs-of-domestic-violence/',
    'https://baylegal.org/get-help/resources/fair-housing-resources/',
    'https://www.hud.gov/program_offices/fair_housing_equal_opp/online-complaint',
    'https://calcivilrights.ca.gov/complaintprocess/',
    'https://www.marincounty.org/-/media/files/departments/da/consumer-guides/tenantshandbook.pdf',
    'https://calcivilrights.ca.gov/',
    'https://evictiondefense.org/',
    



    # Add more starting URLs as needed
]

# Maximum number of pages to scrape per website
max_pages = 5
max_retries = 3

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to scrape content from a given URL with headers
def scrape_website(url):
    try:
        for attempt in range(max_retries):
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()  # Check for request errors
                break  # If the request is successful, exit the retry loop
            except requests.RequestException as e:
                if attempt < max_retries - 1:  # Retry only if attempts are remaining
                    print(f"Retrying {url}: attempt {attempt + 1}")
                    time.sleep(2)  # Wait before retrying
                else:
                    raise  # If no attempts are left, raise the exception
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extracting the text content
        content = soup.get_text(separator=' ', strip=True)
        
        return {
            'url': url,
            'content': content,
            'links': [urljoin(url, a['href']) for a in soup.find_all('a', href=True)]
        }
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return {
            'url': url,
            'content': None,
            'links': [],
            'error': str(e)
        }

# Function to crawl a website starting from the given URL
def crawl_website(start_url, max_pages):
    to_visit = [start_url]
    visited = set()
    website_contents = []

    while to_visit and len(website_contents) < max_pages:
        current_url = to_visit.pop(0)
        if current_url not in visited:
            content_dict = scrape_website(current_url)
            website_contents.append(content_dict)
            visited.add(current_url)
            to_visit.extend([link for link in content_dict['links'] if link not in visited])

            # Delay to prevent overwhelming the server
            time.sleep(1)

    return website_contents

# Scraping each starting URL
all_website_contents = []

for start_url in start_urls:
    website_contents = crawl_website(start_url, max_pages)
    all_website_contents.extend(website_contents)

for w in all_website_contents:
    del w['links']
# Save the contents to a JSON file
with open('website_contents.json', 'w', encoding='utf-8') as json_file:
    json.dump(all_website_contents, json_file, ensure_ascii=False, indent=4)

print("Scraping completed and saved to website_contents.json")


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping completed and saved to website_contents.json
